# Hyperparameter tuning with XGBoost, Ray Tune, Hyperopt and Optuna

## Introduction


In this post we are going to demonstrate how we can speed up hyperparameter tuning with:

1) Bayesian optimization tuning algos like HyperOpt and Optuna, running on…

2) the [Ray](https://ray.io/) distributed ML framework, with a [unified API to many hyperparameter search algos](https://medium.com/riselab/cutting-edge-hyperparameter-tuning-with-ray-tune-be6c0447afdf) and…

3) a distributed cluster of cloud instances for even more speedup.

### Outline:
- Overview of hyperparameter tuning
- Baseline linear regression with no hyperparameters
- ElasticNet with L1 and L2 regularization using ElasticNetCV hyperparameter optimization
- ElasticNet with GridSearchCV hyperparameter optimization
- XGBoost: sequential grid search over hyperparameter subsets with early stopping 
- XGBoost: with HyperOpt and Optuna search algorithms
- LightGBM: with HyperOpt and Optuna search algorithms
- XGBoost: HyperOpt on a Ray cluster
- LightGBM: HyperOpt on a Ray cluster
- Conclusions

But first, here are results on the Ames housing data set, predicting Iowa home prices:

| ML Algo           | Hyperparameter search algo   | CV Error (RMSE in $)  | Time     |
|-------------------|------------------------------|-----------------------|----------|
| XGB               | Sequential Grid Search       | $18783                |   36:09  |
| XGB               | HyperOpt (128 samples)       | $18770                |   14:08  |
| LightGBM          | HyperOpt (128 samples)       | $18770                |   14:08  |
| XGB               | Optuna                       | $18618
| LightGBM          | Optuna                       | $18618
| XGB               | Optuna - 16-instance cluster | $18618
| LightGBM          | Optuna - 16-instance cluster | $18618
| Linear Regression | --                           | $18192                |   0:01s  |
| ElasticNet        | ElasticNetCV (Grid Search)   | $18122                |   0:02s  |          
| ElasticNet        | GridSearchCV                 | $18061                |   0:05s  |          

We see both speedup and RMSE improvement when using HyperOpt and Optuna, and the cluster. But our feature engineering was quite good and our simple linear model still outperforms boosting. (Not shown, SVR and KR are high-performing and an ensemble improves over all individual algos)


## Hyperparameter Tuning Overview

Here are [the principal approaches to hyperparameter tuning](https://en.wikipedia.org/wiki/Hyperparameter_optimization)

- Grid search: given a finite set of discrete values for each hyperparameter, exhaustively cross-validate all combinations

- Random search: given a discrete or continuous distribution for each hyperparameter, randomly sample from the joint distribution. Generally [more efficient than exhaustive grid search.](https://dl.acm.org/doi/10.5555/2188385.2188395 ) 

- Bayesian optimization: update the search space as you go based on outcomes of prior searches.

- Gradient-based optimization: attempt to estimate the gradient of the CV metric with respect to the hyperparameter and ascend/descend the gradient.

- Evolutionary optimization: sample the search space, discard combinations with poor metrics, and genetically evolve new combinations to try based on the successful combinations.

- Population-based: A method of performing hyperparameter optimization at the same time as training.

In this post we focus on Bayesian optimization with HyperOpt and Optuna. What is Bayesian optimization? When we perform a grid search, the search space can be considered a prior belief that the best hyperparameter vector is in the search space, and the combinations have equal probability of being the best combination. So we try them all and pick the best one.

Perhaps we might do two passes of grid search. After an initial search on a broad, coarsely spaced grid, we might do a deeper dive in a smaller area around the best metric from the first pass, with a more finely-spaced grid. In Bayesian terminology we updated our prior belief.

Bayesian optimization first samples randomly, e.g. 30 combinations, and computes the cross-validation metric for each combination. Then the algorithm updates the distribution it samples from, so it is more likely to sample combinations near the good metrics, and less likely to sample combinations near the poor metrics. As it continues to sample, it continues to update the search distribution based on the metrics it finds.

Early stopping may also highly beneficial: often we can discard a combination without fully training it. In this post we use [ASHA](https://arxiv.org/abs/1810.05934). 

We use 4 regression algorithms:
- LinearRegression: baseline with no hyperparameters
- ElasticNet: Linear regression with L1 and L2 regularization (2 hyperparameters).
- XGBoost
- LightGBM

We use 5 approaches :
- *Native CV*: In sklearn if an algo has hyperparameters it will often have an xxxCV version which performs automated hyperparameter tuning over a search space with specified kfolds.
- *GridSearchCV*: Abstracts CV for any sklearn algo, running multithreaded trials over specified folds. 
- *Manual sequential grid search*: What we typically do with XGBoost, which doesn't play well with GridSearchCV and has too many hyperparameters to tune in one pass.
- *Ray on local machine*: HyperOpt and Optuna with early stopping.
- *Ray on cluster*: Additionally scale out to run a single hyperparameter optimization task over many instances.

We use data from the Ames Housing Dataset https://www.kaggle.com/c/house-prices-advanced-regression-techniques . The original data has 79 raw features. The data we will use has 100 features with a fair amount of feature engineering from [my own attempt at modeling](https://github.com/druce/iowa), which was in the top 5% or so when I submitted it to Kaggle.

### Further reading: 
 - [Hyper-Parameter Optimization: A Review of Algorithms and Applications](https://arxiv.org/abs/2003.05689) Tong Yu, Hong Zhu (2020)
 - [Hyperparameter Search in Machine Learning](https://arxiv.org/abs/1502.02127v2), Marc Claesen, Bart De Moor (2015)
 - [Hyperparameter Optimization](https://link.springer.com/chapter/10.1007/978-3-030-05318-5_1), Matthias Feurer, Frank Hutter (2019) 

In [1]:
from itertools import product
from datetime import datetime, timedelta
import os
import random
import string

import numpy as np
import pandas as pd

import sklearn
from sklearn.linear_model import LinearRegression, ElasticNet, ElasticNetCV, Ridge, RidgeCV
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV, KFold
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline

#!conda install -y -c conda-forge  xgboost 
import xgboost
from xgboost import XGBRegressor
from xgboost import plot_importance

import lightgbm
from lightgbm import LGBMRegressor

import ray
from ray import tune
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.bayesopt import BayesOptSearch
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.suggest.optuna import OptunaSearch
from ray.tune.logger import DEFAULT_LOGGERS
from ray.tune.integration.wandb import WandbLogger

# pip install hyperopt
# pip install optuna

import wandb
os.environ['WANDB_NOTEBOOK_NAME']='hyperparameter_optimization.ipynb'

print(datetime.now())

print ("%-20s %s"% ("numpy", np.__version__))
print ("%-20s %s"% ("pandas", pd.__version__))
print ("%-20s %s"% ("sklearn", sklearn.__version__))
print ("%-20s %s"% ("xgboost", xgboost.__version__))
print ("%-20s %s"% ("lightgbm", lightgbm.__version__))
print ("%-20s %s"% ("ray", ray.__version__))


2020-10-14 02:20:02.179813
numpy                1.19.1
pandas               1.1.3
sklearn              0.23.2
xgboost              1.2.0
lightgbm             2.3.0
ray                  1.0.0


In [2]:
# set seed for reproducibility
RANDOMSTATE = 42
np.random.seed(RANDOMSTATE)


In [3]:
def get_random_tag(length):
    """random tag for experiments"""
    letters_and_digits = string.ascii_letters + string.digits
    result_str = ''.join((random.choice(letters_and_digits) for i in range(length)))
    return result_str.upper()

get_random_tag(8)

'ZY4HMFBQ'

In [4]:
# import train data
df = pd.read_pickle('df_train.pickle')

response = 'SalePrice'
predictors = ['YearBuilt',
              'BsmtFullBath',
              'FullBath',
              'KitchenAbvGr',
              'GarageYrBlt',
              'LotFrontage',
              'MasVnrArea',
              '1stFlrSF',
              'GrLivArea',
              'GarageArea',
              'WoodDeckSF',
              'PorchSF',
              'AvgBltRemod',
              'FireBathRatio',
              'TotalSF x OverallQual x OverallCond',
              'AvgBltRemod x Functional x TotalFinSF',
              'Functional x OverallQual',
              'KitchenAbvGr x KitchenQual',
              'GarageCars x GarageYrBlt',
              'GarageQual x GarageCond x GarageCars',
              'HeatingQC x Heating',
              'monthnum',
              'log_YearBuilt',
              'log_LotArea',
              'log_TotalFinSF',
              'log_GarageRatio',
              'log_TotalSF x OverallQual x OverallCond',
              'log_TotalSF x OverallCond',
              'log_AvgBltRemod x TotalFinSF',
              'sq_2ndFlrSF',
              'sq_BsmtFinSF',
              'sq_BsmtFinSF x BsmtQual',
              'sq_BsmtFinSF x BsmtBath',
              'BldgType_4',
              'BsmtExposure_1',
              'BsmtExposure_4',
              'BsmtFinType1_1',
              'BsmtFinType1_2',
              'BsmtFinType1_4',
              'BsmtFinType1_5',
              'BsmtFinType1_6',
              'CentralAir_0',
              'CentralAir_1',
              'Condition1_1',
              'Condition1_3',
              'ExterCond_2',
              'ExterQual_2',
              'Exterior1st_4',
              'Exterior1st_5',
              'Exterior1st_10',
              'Fence_0',
              'Fence_2',
              'Foundation_1',
              'Foundation_5',
              'GarageCars_1',
              'GarageFinish_2',
              'GarageFinish_3',
              'GarageType_2',
              'HouseStyle_2',
              'KitchenQual_4',
              'LotConfig_0',
              'LotConfig_4',
              'MSSubClass_30',
              'MSSubClass_70',
              'MSZoning_0',
              'MSZoning_1',
              'MSZoning_4',
              'MasVnrType_2',
              'MasVnrType_3',
              'MoSold_1',
              'MoSold_5',
              'MoSold_6',
              'MoSold_11',
              'Neighborhood_3',
              'Neighborhood_4',
              'Neighborhood_5',
              'Neighborhood_10',
              'Neighborhood_11',
              'Neighborhood_16',
              'Neighborhood_17',
              'Neighborhood_19',
              'Neighborhood_22',
              'Neighborhood_24',
              'OverallCond_7',
              'OverallQual_5',
              'OverallQual_6',
              'OverallQual_7',
              'OverallQual_9',
              'PavedDrive_0',
              'PavedDrive_2',
              'SaleCondition_1',
              'SaleCondition_2',
              'SaleCondition_5',
              'SaleType_4',
              'BedroomAbvGr_1',
              'BedroomAbvGr_4',
              'BedroomAbvGr_5',
              'HalfBath_1',
              'TotalBath_1.0',
              'TotalBath_2.5']

X_train, X_test, y_train, y_test = train_test_split(df, df[response], test_size=.25)

display(df[predictors].head())
display(df[[response]].head())


,YearBuilt,BsmtFullBath,FullBath,KitchenAbvGr,GarageYrBlt,LotFrontage,MasVnrArea,1stFlrSF,GrLivArea,GarageArea,...,SaleCondition_1,SaleCondition_2,SaleCondition_5,SaleType_4,BedroomAbvGr_1,BedroomAbvGr_4,BedroomAbvGr_5,HalfBath_1,TotalBath_1.0,TotalBath_2.5
Id,,,,,,,,,,,,,,,,,,,,,
1,7,1,2,1,7,65.0,196.0,856,1710,548.0,...,0,0,0,1,0,0,0,1,0,0
2,34,0,2,1,34,80.0,0.0,1262,1262,460.0,...,0,0,0,1,0,0,0,0,0,1
3,9,1,2,1,9,68.0,162.0,920,1786,608.0,...,0,0,0,1,0,0,0,1,0,0
4,95,1,1,1,12,60.0,0.0,961,1717,642.0,...,1,0,0,1,0,0,0,0,0,0
5,10,1,2,1,10,84.0,350.0,1145,2198,836.0,...,0,0,0,1,0,1,0,1,0,0


,SalePrice
Id,
1,12.247699
2,12.109016
3,12.317171
4,11.849405
5,12.429220


In [5]:
# we are training on a response which is the log of 1 + the sale price
# transform prediction back to original basis with expm1 and evaluate vs. original

def evaluate(y_train, y_pred_train, y_test, y_pred_test):
    """evaluate in train_test split"""
    print('Train RMSE', np.sqrt(mean_squared_error(np.expm1(y_train), np.expm1(y_pred_train))))
    print('Train R-squared', r2_score(np.expm1(y_train), np.expm1(y_pred_train)))
    print('Train MAE', mean_absolute_error(np.expm1(y_train), np.expm1(y_pred_train)))
    print()
    print('Test RMSE', np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(y_pred_test))))
    print('Test R-squared', r2_score(np.expm1(y_test), np.expm1(y_pred_test)))
    print('Test MAE', mean_absolute_error(np.expm1(y_test), np.expm1(y_pred_test)))

MEAN_RESPONSE=df[response].mean()
def cv_to_raw(cv_val, mean_response=MEAN_RESPONSE):
    """convert log1p rmse to underlying SalePrice error"""
    # MEAN_RESPONSE assumes folds have same mean response, which is true in expectation but not in each fold
    # we can also pass the actual response for each fold
    # but we're usually looking to consistently convert the log value to a more meaningful unit
    return np.expm1(mean_response+cv_val) - np.expm1(mean_response)

In [6]:
# always use same k-folds for reproducibility
kfolds = KFold(n_splits=10, shuffle=True, random_state=RANDOMSTATE)


In [10]:
# refactor to give ray.tune a single function of hyperparameters to optimize
def my_xgb(config):
    
    # fix these configs 
    config['max_depth'] += 2   # hyperopt needs left to start at 0 but we want to start at 2
    config['max_depth'] = int(config['max_depth'])
    config['n_estimators'] = int(config['n_estimators'])   # pass float eg loguniform distribution, use int
    
    xgb = XGBRegressor(
        objective='reg:squarederror',
        n_jobs=1,
        random_state=RANDOMSTATE,
        booster='gbtree',   
        base_score=0.5, 
        scale_pos_weight=1, 
        **config,
    )
    scores = np.sqrt(-cross_val_score(xgb, df[predictors], df[response],
                                      scoring="neg_mean_squared_error",
                                      cv=kfolds))
    tune.report(mse=np.mean(scores))
    return {'mse': np.mean(scores)}


# Ray Cluster

- Cluster config is in `ray1.1.yaml`
- Edit `ray1.1.yaml` file with your region, availability zone, subnet, imageid information
    - to get those variables launch the latest Deep Learning AMI (Ubuntu 18.04) Version 35.0 into a small instance in your favorite region/zone
    - test that it works
    - note those 4 variables: region, availability zone, subnet, AMI imageid
    - terminate the instance and edit `ray1.1.yaml` accordingly
    - in future you can create your own image with everything pre-installed and specify its AMI imageid, instead of using the generic image and installing everything at launch.
- To run the cluster: 
`ray up ray1.1.yaml`
    - Creates head instance using image specified.
    - Installs ray and related requirements
    - Clones this Iowa repo
    - Launches worker nodes per auto-scaling parameters (currently we fix the number of nodes because we're not benching the time the cluster will take to auto-scale)
- After cluster starts you can check AWS console and note that several instances launched.
- Check `ray monitor ray1.1.yaml` for any error messages
- Run Jupyter on the cluster with port forwarding
 `ray exec ray1.1.yaml --port-forward=8899 'jupyter notebook --port=8899'`
- Open the notebook on the generated URL e.g. http://localhost:8899/?token=5f46d4355ae7174524ba71f30ef3f0633a20b19a204b93b4
- Make sure to hoose the default kernel to make sure it runs in the conda environment with all installs
- Make sure to use the ray.init() command given in the startup messages.
- You can also run a terminal on the head node of the cluster with
 `ray attach /Users/drucev/projects/iowa/ray1.1.yaml`
- You can also ssh explicitly with the IP address and the generated private key
 `ssh -o IdentitiesOnly=yes -i ~/.ssh/ray-autoscaler_1_us-east-1.pem ubuntu@54.161.200.54`
- run port forwarding to the Ray dashboard with   
`ray dashboard ray1.1.yaml`
and then open
 http://localhost:8265/

see https://docs.ray.io/en/latest/cluster/launcher.html for additional info

In [11]:
# make sure local ray service is shutdown
ray.shutdown()


In [12]:
# launch cluster in terminal
# initialize ray on cluster
ray.init(address='localhost:6379', _redis_password='5241590000000000')


2020-10-14 02:21:04,456	INFO worker.py:634 -- Connecting to existing Ray cluster at address: 172.30.1.147:6379


{'node_ip_address': '172.30.1.147',
 'raylet_ip_address': '172.30.1.147',
 'redis_address': '172.30.1.147:6379',
 'object_store_address': '/tmp/ray/session_2020-10-14_02-12-25_864293_32111/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-10-14_02-12-25_864293_32111/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-10-14_02-12-25_864293_32111',
 'metrics_export_port': 60373}

In [13]:
# run hyperopt/xgb on cluster
NUM_SAMPLES=256

start_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))

algo = HyperOptSearch(random_state_seed=RANDOMSTATE)
# uncomment and set max_concurrent to limit number of cores
# algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

tune_kwargs = {
    "num_samples": NUM_SAMPLES,
    "config": {
        "n_estimators": tune.loguniform(100, 10000),
        "max_depth": tune.randint(0, 6),
        'min_child_weight': tune.randint(0, 6),
        "subsample": tune.quniform(0.4, 0.9, 0.05),
        "colsample_bytree": tune.quniform(0.05, 0.8, 0.05),
        "reg_alpha": tune.loguniform(1e-04, 1),
        "reg_lambda": tune.loguniform(1e-04, 100),
        "gamma": 0,
        "learning_rate": tune.loguniform(0.001, 0.1),
        "wandb": {
            "project": "iowa",
            "api_key_file": "secrets/wandb.txt",
            "log_config": True
        }    
    }
}

analysis = tune.run(my_xgb,
                    name="xgb_hyperopt",
                    metric="mse",
                    mode="min",
                    search_alg=algo,
                    scheduler=scheduler,
                    verbose=1, 
                    raise_on_failed_trial=False, # otherwise no reults df returned if any trial error
                    loggers=DEFAULT_LOGGERS + (WandbLogger, ),
                    **tune_kwargs)

end_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))
print("%-20s %s" % ("End Time", end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))

Trial name,status,loc,colsample_bytree,gamma,learning_rate,max_depth,min_child_weight,n_estimators,reg_alpha,reg_lambda,subsample,wandb/api_key_file,wandb/log_config,wandb/project,iter,total time (s),mse
my_xgb_f61ca264,TERMINATED,,0.7,0,0.0105391,0,4,435.406,0.000188888,9.90358,0.65,secrets/wandb.txt,True,iowa,1,15.3225,0.203631
my_xgb_f61f3bc8,TERMINATED,,0.3,0,0.00673344,3,2,428.153,0.538855,2.32463,0.6,secrets/wandb.txt,True,iowa,1,10.0508,0.681993
my_xgb_f6215886,TERMINATED,,0.15,0,0.00133237,5,4,7988.18,0.00163546,19.9476,0.8,secrets/wandb.txt,True,iowa,1,238.506,0.114781
my_xgb_f6236734,TERMINATED,,0.2,0,0.0187659,2,3,735.64,0.000262686,32.5338,0.6,secrets/wandb.txt,True,iowa,1,18.2675,0.11581
my_xgb_f6258b68,TERMINATED,,0.6,0,0.00155207,0,3,4032.08,0.0246711,0.00107456,0.8,secrets/wandb.txt,True,iowa,1,124.414,0.124603
my_xgb_f6279426,TERMINATED,,0.15,0,0.00106609,1,5,2700.39,0.0127785,0.0424791,0.8,secrets/wandb.txt,True,iowa,1,50.4484,0.661069
my_xgb_f629b2f6,TERMINATED,,0.4,0,0.00556109,3,2,238.15,0.000451262,43.2731,0.75,secrets/wandb.txt,True,iowa,1,4.59826,3.25644
my_xgb_f62bcda2,TERMINATED,,0.55,0,0.0154611,0,1,5605.97,0.00272955,4.48938,0.7,secrets/wandb.txt,True,iowa,1,164.448,0.109036
my_xgb_f62ee6e0,TERMINATED,,0.5,0,0.00126552,5,3,4734.92,0.00967981,33.1173,0.65,secrets/wandb.txt,True,iowa,1,219.855,0.1519
my_xgb_f6317658,TERMINATED,,0.75,0,0.00153686,2,2,678.644,0.306625,1.59856,0.9,secrets/wandb.txt,True,iowa,1,22.5847,4.08285


Start Time           2020-10-14 02:21:26.313456
End Time             2020-10-14 02:35:49.425682
0:14:23


In [14]:
analysis_results_df = analysis.results_df[['mse', 'date', 'time_this_iter_s',
       'config.n_estimators', 'config.max_depth', 'config.min_child_weight', 'config.subsample',
       'config.colsample_bytree', 'config.reg_alpha', 'config.reg_lambda', 'config.gamma',
       'config.learning_rate']].sort_values('mse')
analysis_results_df


,mse,date,time_this_iter_s,config.n_estimators,config.max_depth,config.min_child_weight,config.subsample,config.colsample_bytree,config.reg_alpha,config.reg_lambda,config.gamma,config.learning_rate
trial_id,,,,,,,,,,,,
f6437f06,0.106830,2020-10-14_02-23-02,78.898158,4374,4,3,0.75,0.10,0.009577,1.219649,0,0.023078
f7846344,0.106832,2020-10-14_02-24-54,136.299327,7221,3,3,0.65,0.15,0.013007,1.561307,0,0.009488
f789a6e2,0.106860,2020-10-14_02-23-58,80.995363,5468,3,4,0.80,0.05,0.072716,2.172529,0,0.007060
fd0337dc,0.107215,2020-10-14_02-31-24,90.287300,5982,3,4,0.60,0.05,0.028824,7.170552,0,0.009774
f84654cc,0.107370,2020-10-14_02-25-08,78.314308,6130,3,4,0.80,0.05,0.024101,5.281423,0,0.012514
...,...,...,...,...,...,...,...,...,...,...,...,...
f7dac018,6.709540,2020-10-14_02-23-29,13.181123,213,6,5,0.80,0.65,0.230677,0.000495,0,0.002540
fae366d4,6.890884,2020-10-14_02-27-46,19.443758,320,5,5,0.80,0.70,0.136117,0.000365,0,0.001608
f8c596d8,7.572926,2020-10-14_02-24-38,10.731139,203,6,5,0.80,0.70,0.331362,0.000547,0,0.002070


In [15]:
max_depth = analysis_results_df.iloc[0]['config.max_depth']
min_child_weight = analysis_results_df.iloc[0]['config.min_child_weight']
subsample = analysis_results_df.iloc[0]['config.subsample']
colsample_bytree = analysis_results_df.iloc[0]['config.colsample_bytree']
reg_alpha = analysis_results_df.iloc[0]['config.reg_alpha']
reg_lambda = analysis_results_df.iloc[0]['config.reg_lambda']
reg_gamma = analysis_results_df.iloc[0]['config.gamma']
learning_rate = analysis_results_df.iloc[0]['config.learning_rate']
N_ESTIMATORS = analysis_results_df.iloc[0]['config.n_estimators']    

best_config = {
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    'reg_alpha': reg_alpha,
    'reg_lambda': reg_lambda,
    'gamma': reg_gamma,
    'learning_rate': learning_rate,
    'n_estimators':  N_ESTIMATORS
}

xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=RANDOMSTATE,    
    verbosity=1,
    n_jobs=-1,
    **best_config
)
print(xgb)

scores = -cross_val_score(xgb, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds)

raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.06f (STD %.04f)" % (np.mean(raw_scores), np.std(raw_scores)))


XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.1, gamma=0, gpu_id=None,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.023077837289899438, max_delta_step=None,
             max_depth=4, min_child_weight=3, missing=nan,
             monotone_constraints=None, n_estimators=4374, n_jobs=-1,
             num_parallel_tree=None, random_state=42,
             reg_alpha=0.009577332554566264, reg_lambda=1.2196486740226578,
             scale_pos_weight=None, subsample=0.75, tree_method=None,
             validate_parameters=None, verbosity=1)

Log1p CV RMSE 0.106830 (STD 0.0120)
Raw CV RMSE 18770.443039 (STD 2229.3542)


In [17]:
#lgbm on cluster
print("LightGBM")

NUM_SAMPLES=256

start_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))

def my_lgbm(config):
    
    # fix these configs 
    config['n_estimators'] = int(config['n_estimators'])   # pass float eg loguniform distribution, use int
    config['num_leaves'] = 2 + int(config['num_leaves'])
    
    lgbm = LGBMRegressor(objective='regression',
                         num_leaves=config['num_leaves'],
                         learning_rate=config['learning_rate'],
                         n_estimators=config['n_estimators'],
                         max_bin=200,
                         bagging_fraction=config['bagging_fraction'],
                         feature_fraction=config['feature_fraction'],
                         feature_fraction_seed=7,
                         min_data_in_leaf=2,
                         # these are to suppress warnings
                         colsample_bytree=None,
                         min_child_samples=None,
                         subsample=None,
                         verbose=-1,
                         # early stopping params, maybe in fit
                         #early_stopping_rounds=early_stopping_rounds,
                         #valid_sets=[xgtrain, xgvalid], valid_names=['train','valid'], evals_result=evals_results
                         #num_boost_round=num_boost_round,
                         )
    
    scores = np.sqrt(-cross_val_score(lgbm, df[predictors], df[response],
                                      scoring="neg_mean_squared_error",
                                      cv=kfolds))
    tune.report(mse=np.mean(scores))
    return {'mse': np.mean(scores)}

algo = HyperOptSearch(random_state_seed=RANDOMSTATE)
# uncomment and set max_concurrent to limit number of cores
# algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

tune_kwargs = {
    "num_samples": NUM_SAMPLES,
    "config": {
        "n_estimators": tune.loguniform(100, 10000),
        'num_leaves': tune.randint(0, 10),
        "bagging_fraction": tune.uniform(0.5, 0.8),
        "feature_fraction": tune.uniform(0.01, 0.8),
        "learning_rate": tune.loguniform(0.001, 0.1),
        "wandb": {
            "project": "iowa",
            "api_key_file": "secrets/wandb.txt",
            "log_config": True
        }    
    }
}

analysis = tune.run(my_lgbm,
                    name="lgbm_hyperopt",
                    metric="mse",
                    mode="min",
                    search_alg=algo,
                    scheduler=scheduler,
                    verbose=1,
                    raise_on_failed_trial=False, # otherwise no reults df returned if any trial error                    
                    loggers=DEFAULT_LOGGERS + (WandbLogger, ),
                    **tune_kwargs)

end_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))
print("%-20s %s" % ("End Time", end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))


Trial name,status,loc,bagging_fraction,feature_fraction,learning_rate,n_estimators,num_leaves,wandb/api_key_file,wandb/log_config,wandb/project,iter,total time (s),mse
my_lgbm_32c0ea7e,TERMINATED,,0.645556,0.262363,0.00137437,4626.62,9,secrets/wandb.txt,True,iowa,1,57.675,0.113285
my_lgbm_32c2b80e,TERMINATED,,0.701044,0.259482,0.0734068,2853.98,7,secrets/wandb.txt,True,iowa,1,25.0598,0.109697
my_lgbm_32c4da80,TERMINATED,,0.668672,0.484699,0.0188581,1180.42,6,secrets/wandb.txt,True,iowa,1,15.1347,0.111762
my_lgbm_32c703c8,TERMINATED,,0.540469,0.0265392,0.01101,191.949,0,secrets/wandb.txt,True,iowa,1,0.609982,0.2627
my_lgbm_32c8c4ba,TERMINATED,,0.528971,0.0457216,0.091503,905.561,6,secrets/wandb.txt,True,iowa,1,3.47128,0.110973
my_lgbm_32ca5fd2,TERMINATED,,0.781842,0.599419,0.00253267,494.528,5,secrets/wandb.txt,True,iowa,1,7.18635,0.194386
my_lgbm_32cbf81a,TERMINATED,,0.794835,0.45121,0.0400602,7345.14,3,secrets/wandb.txt,True,iowa,1,89.5243,0.111324
my_lgbm_32cd50ac,TERMINATED,,0.616283,0.541449,0.0027847,395.202,6,secrets/wandb.txt,True,iowa,1,6.67093,0.203936
my_lgbm_32ceb212,TERMINATED,,0.748052,0.787677,0.00167074,185.032,8,secrets/wandb.txt,True,iowa,1,4.87512,0.313964
my_lgbm_32d025fc,TERMINATED,,0.688821,0.361669,0.00933932,897.568,8,secrets/wandb.txt,True,iowa,1,11.6028,0.113955


Start Time           2020-10-14 02:44:36.572769
End Time             2020-10-14 02:48:58.610025
0:04:22


In [18]:
analysis_results_df = analysis.results_df[['mse', 'date', 'time_this_iter_s',
       'config.n_estimators', 'config.num_leaves', 'config.bagging_fraction',
       'config.feature_fraction', 'config.learning_rate']].sort_values('mse')
analysis_results_df


,mse,date,time_this_iter_s,config.n_estimators,config.num_leaves,config.bagging_fraction,config.feature_fraction,config.learning_rate
trial_id,,,,,,,,
335d0a80,0.105969,2020-10-14_02-45-08,9.581070,2758,5,0.754804,0.049949,0.027701
336fd5f2,0.106189,2020-10-14_02-45-13,10.220761,1742,10,0.517380,0.055519,0.020289
33b3654c,0.106252,2020-10-14_02-45-28,9.276073,2715,5,0.700356,0.043747,0.026954
3425fa80,0.106657,2020-10-14_02-45-50,5.669229,1758,5,0.699876,0.039295,0.045055
3507f0ca,0.106706,2020-10-14_02-46-34,5.987449,1734,5,0.712548,0.047620,0.043218
...,...,...,...,...,...,...,...,...
33d7f40c,0.323659,2020-10-14_02-45-32,1.978369,247,11,0.724945,0.102919,0.001350
345a4b82,0.344946,2020-10-14_02-45-57,1.642225,153,11,0.725653,0.170354,0.001309
356db3f6,0.346311,2020-10-14_02-46-49,3.838375,155,11,0.681775,0.646190,0.001111


In [19]:
num_leaves = analysis_results_df.iloc[0]['config.num_leaves']
bagging_fraction = analysis_results_df.iloc[0]['config.bagging_fraction']
feature_fraction = analysis_results_df.iloc[0]['config.feature_fraction']
learning_rate = analysis_results_df.iloc[0]['config.learning_rate']
N_ESTIMATORS = analysis_results_df.iloc[0]['config.n_estimators']    

best_config = {
    'num_leaves': num_leaves,
    'bagging_fraction': bagging_fraction,
    'feature_fraction': feature_fraction,
    'learning_rate': learning_rate,
    'n_estimators':  N_ESTIMATORS
}

lgbm = LGBMRegressor(objective='regression',
                     num_leaves=best_config['num_leaves'],
                     learning_rate=best_config['learning_rate'],
                     n_estimators=best_config['n_estimators'],
                     max_bin=200,
                     bagging_fraction=best_config['bagging_fraction'],
                     feature_fraction=best_config['feature_fraction'],
                     feature_fraction_seed=7,
                     min_data_in_leaf=2,
                     verbose=-1,
                     # early stopping params, maybe in fit
                     #early_stopping_rounds=early_stopping_rounds,
                     #valid_sets=[xgtrain, xgvalid], valid_names=['train','valid'], evals_result=evals_results
                     #num_boost_round=num_boost_round,
                     )
 
print(lgbm)

scores = -cross_val_score(lgbm, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds)

raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.06f (STD %.04f)" % (np.mean(raw_scores), np.std(raw_scores)))
raw_scores = [cv_to_raw(x) for x in scores]

LGBMRegressor(bagging_fraction=0.7548044328777966,
              feature_fraction=0.04994854048380451, feature_fraction_seed=7,
              learning_rate=0.027701226718815856, max_bin=200,
              min_data_in_leaf=2, n_estimators=2758, num_leaves=5,
              objective='regression', verbose=-1)

Log1p CV RMSE 0.105969 (STD 0.0126)
Raw CV RMSE 18612.385681 (STD 2325.3205)


## Concluding remarks

We observe a modest but non-negligeable improvement in the target metric with a less manual process vs. sequential tuning.

I'll be using HyperOpt and Optuna for xgboost going forward, no more grid search for me. In every case I've applied them, I've gotten at least a small improvement in the best metrics I found using grid search methods. Additionally, it's fire and forget (although with a little elbow grease the 4-pass sequential grid search could be made fire and forget.)

These two algorithms seem to be the most popular but I may try the other algos systematically. 

I was surprised that Elasticnet, i.e. regularized linear regression outperforms boosting. 
This dataset has been heavily engineered so that linear methods work well. Predictors were chosen using lasso/elasticnet and we used log and Box-Cox transforms to force predictors to follow assumptions of least-squares.  

This tends to validate one of the [critiques of machine learning](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3624052), that the most powerful ML methods don't necessarily converge all the way to the best solution. If you have a ground truth that is linear plus noise, a complex XGBoost or neural network algorithm should get arbitrarily close to a closed-form solution but will never match it exactly due to complexity of the algorithm. XGBoost is piecewise constant and the complex neural network is subject to the vagaries of stochastic gradient descent. 

But Elasticnet with L1 + L2 regularization plus gradient descent and hyperparameter optimization is still machine learning, simply the form best matched to the problem. In the real world where things don't match assumptions of least-squares, boosting performs extremely well. And even on this dataset, engineered for the linear models, ensembling Elasticnet with XGBoost, LightGBM, SVR, neural networks works best of all. 

To paraphrase Casey Stengel, clever feature engineering will always outperform clever model algorithms and vice-versa<sup>*</sup>. But improving hyperparameters will always improve your results.

<sup>*</sup>This is not intended to make sense.
